In [1]:
import requests
import pandas as pd
import numpy as np
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split


In [2]:
import tensorflow.compat.v1 as tf
from keras.models import Model, Sequential
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.metrics import CategoricalAccuracy
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input as preprocessing_function


In [3]:
classes = { 0:'5_18',
            1:'3_24', 
            2:'1_16', 
            3:'5_15_5', 
            4:'5_19_1', 
            5:'5_19_2', 
            6:'1_20_1', 
            7:'8_23', 
            8:'2_1', 
            9:'4_2_1', 
            10:'8_22_1', 
            11:'6_16', 
            12:'1_22', 
            13:'1_2', 
            14:'5_16', 
            15:'3_27', 
            16:'6_10_1', 
            17:'8_2_4', 
            18:'6_12', 
            19:'5_15_2', 
            20:'3_13', 
            21:'3_1', 
            22:'3_20', 
            23:'3_12', 
            24:'7_14_2', 
            25:'5_23_1', 
            26:'2_4', 
            27:'5_6', 
            28:'4_2_3', 
            29:'8_22_3', 
            30:'5_15_1',
            31: '7_3', 
            32:'3', 
            33:'2_3_1', 
            34:'3_11', 
            35:'6_13', 
            36:'5_15_4', 
            37:'8_2_1', 
            38:'1_34_3', 
            39:'8_2_2', 
            40:'5_15_3', 
            41:'1_17', 
            42:'4_1_1',
            43: '4_1_4',
            44: '3_25',
            45: '1_20_2',
            46: '8_22_2',
            47: '6_9_2',
            48: '3_2',
            49: '5_5',
            50: '5_15_7',
            51: '7_12',
            52: '8_2_3',
            53: '5_24_1',
            54: '1_25',
            55: '3_28',
            56: '5_9_1',
            57: '5_15_6',
            58: '8_1_1',
            59: '1_10',
            60: '6_11',
            61: '3_4',
            62: '6_10',
            63: '6_9_1',
            64: '8_2_5',
            65: '5_15',
            66: '4_8_2',
            67: '8_22',
            68: '5_21'}

In [4]:
train_data = pd.read_csv('data/trainset/train.csv')
test_data = pd.read_csv('data/testset/test.csv')

In [5]:
sings = ['sing1', 'sing2', 'sing3', 'sing4', 'sing5', 'sing6', 'sing7', 'sing8']

In [6]:
train_data

,id,img,sing1,sing2,sing3,sing4,sing5,sing6,sing7,sing8
0,807,5-avi-frame24_jpg.rf.5dec372f9195e9a88ff7dd3bd...,1,2,37,0,0,0,0,0
1,121,6-avi-frame6431_jpg.rf.1ad48ac0ce545b88cefb946...,8,0,0,0,0,0,0,0
2,1130,9-avi-frame1457_jpg.rf.634a979898a9caa4d106913...,21,0,0,0,0,0,0,0
3,378,3-avi-frame499_jpg.rf.a0464ac21591528194da5bcf...,5,4,0,0,0,0,0,0
4,1079,5-avi-frame5306_jpg.rf.8cd8b5643eea33f5f33714e...,48,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...
773,777,6-avi-frame4576_jpg.rf.8780eec122abf32efb3744c...,8,0,0,0,0,0,0,0
774,739,5-avi-frame3664_jpg.rf.71c9b545a181651166de56a...,8,0,0,0,0,0,0,0
775,873,6-avi-frame6490_jpg.rf.b19dac7915db559d41e7e27...,8,4,5,0,0,0,0,0
776,173,5-avi-frame4887_jpg.rf.9a87ce8aac43977d561d819...,38,51,0,0,0,0,0,0


In [7]:
train_data.head(40)

,id,img,sing1,sing2,sing3,sing4,sing5,sing6,sing7,sing8
0,807,5-avi-frame24_jpg.rf.5dec372f9195e9a88ff7dd3bd...,1,2,37,0,0,0,0,0
1,121,6-avi-frame6431_jpg.rf.1ad48ac0ce545b88cefb946...,8,0,0,0,0,0,0,0
2,1130,9-avi-frame1457_jpg.rf.634a979898a9caa4d106913...,21,0,0,0,0,0,0,0
3,378,3-avi-frame499_jpg.rf.a0464ac21591528194da5bcf...,5,4,0,0,0,0,0,0
4,1079,5-avi-frame5306_jpg.rf.8cd8b5643eea33f5f33714e...,48,0,0,0,0,0,0,0
5,860,6-avi-frame3779_jpg.rf.14ff604c1af34671e5748ab...,4,5,14,0,0,0,0,0
6,769,5-avi-frame16_jpg.rf.77026200267155755337fb456...,2,1,37,0,0,0,0,0
7,298,5-avi-frame1531_jpg.rf.54c3c024dcf98b542ba9446...,1,2,0,0,0,0,0,0
8,500,5-avi-frame3315_jpg.rf.78b197fe5bd8242fbe4c761...,32,4,5,0,0,0,0,0
9,450,5-avi-frame3463_jpg.rf.a37b3477e30833ce9b5c749...,13,0,0,0,0,0,0,0


In [8]:
train_data.drop(['sing1', 'sing2', 'sing3', 'sing4', 'sing5', 'sing6', 'sing7', 'sing8'], axis=1)

,id,img
0,807,5-avi-frame24_jpg.rf.5dec372f9195e9a88ff7dd3bd...
1,121,6-avi-frame6431_jpg.rf.1ad48ac0ce545b88cefb946...
2,1130,9-avi-frame1457_jpg.rf.634a979898a9caa4d106913...
3,378,3-avi-frame499_jpg.rf.a0464ac21591528194da5bcf...
4,1079,5-avi-frame5306_jpg.rf.8cd8b5643eea33f5f33714e...
...,...,...
773,777,6-avi-frame4576_jpg.rf.8780eec122abf32efb3744c...
774,739,5-avi-frame3664_jpg.rf.71c9b545a181651166de56a...
775,873,6-avi-frame6490_jpg.rf.b19dac7915db559d41e7e27...
776,173,5-avi-frame4887_jpg.rf.9a87ce8aac43977d561d819...


In [9]:
class_index = ['sing1', 'sing2', 'sing3', 'sing4', 'sing5', 'sing6', 'sing7', 'sing8']

simple_array = []
chunked_list= list()
chunk_size = 8


for i in range(777):
    for k in class_index:
        simple_array.append(train_data[k][i])
        
for i in range(0, len(simple_array), chunk_size):
    chunked_list.append(simple_array[i:i+chunk_size])

In [10]:
final_list = []

In [11]:
for i in chunked_list:
    i = [k for k in i if k != 0] 
    final_list.append(i)

In [12]:
final_list

[[1, 2, 37],
 [8],
 [21],
 [5, 4],
 [48],
 [4, 5, 14],
 [2, 1, 37],
 [1, 2],
 [32, 4, 5],
 [13],
 [1, 7],
 [8],
 [25],
 [32, 57, 5, 4],
 [15],
 [26],
 [1, 7],
 [8],
 [6],
 [15, 35, 17],
 [8],
 [8],
 [12, 1],
 [15, 35, 17],
 [51],
 [6],
 [8],
 [8, 49, 18],
 [42, 16, 21],
 [15],
 [8, 35],
 [15],
 [1, 15, 2],
 [8, 14],
 [6, 10],
 [4, 15],
 [8, 16],
 [1, 45, 4, 5, 46],
 [8],
 [15, 17],
 [1, 7, 10],
 [2, 1, 37],
 [20, 4, 5],
 [9, 47, 10],
 [1, 2, 37],
 [8, 4, 5],
 [26],
 [6, 55],
 [9, 10, 4, 14],
 [1],
 [8, 51, 38],
 [36],
 [5, 4],
 [8, 4, 5],
 [15, 10],
 [8, 4, 5],
 [5, 4, 20],
 [48],
 [1, 10],
 [4, 5],
 [8, 4],
 [28, 29],
 [8],
 [15],
 [8, 5, 4],
 [8, 35],
 [8, 35],
 [8, 14],
 [8, 4, 5, 16],
 [8],
 [1, 2, 37],
 [28, 29, 30],
 [9, 10],
 [42, 16, 21],
 [8],
 [2],
 [28, 29],
 [9, 10, 11, 4, 5, 14],
 [8, 16, 54],
 [1, 2, 37],
 [4, 5],
 [28, 29],
 [9, 4, 5, 10, 14],
 [8, 14],
 [42],
 [8, 51, 38],
 [6],
 [1, 7],
 [30, 40],
 [4, 5],
 [8, 10, 19, 11],
 [16, 10],
 [5, 4],
 [1, 2],
 [61],
 [8, 4, 5

In [13]:
train_data.drop(['sing1', 'sing2', 'sing3', 'sing4', 'sing5', 'sing6', 'sing7', 'sing8'], axis=1)

,id,img
0,807,5-avi-frame24_jpg.rf.5dec372f9195e9a88ff7dd3bd...
1,121,6-avi-frame6431_jpg.rf.1ad48ac0ce545b88cefb946...
2,1130,9-avi-frame1457_jpg.rf.634a979898a9caa4d106913...
3,378,3-avi-frame499_jpg.rf.a0464ac21591528194da5bcf...
4,1079,5-avi-frame5306_jpg.rf.8cd8b5643eea33f5f33714e...
...,...,...
773,777,6-avi-frame4576_jpg.rf.8780eec122abf32efb3744c...
774,739,5-avi-frame3664_jpg.rf.71c9b545a181651166de56a...
775,873,6-avi-frame6490_jpg.rf.b19dac7915db559d41e7e27...
776,173,5-avi-frame4887_jpg.rf.9a87ce8aac43977d561d819...


In [14]:
train_data['sing_class'] = pd.Series(final_list)

In [15]:
train_data

,id,img,sing1,sing2,sing3,sing4,sing5,sing6,sing7,sing8,sing_class
0,807,5-avi-frame24_jpg.rf.5dec372f9195e9a88ff7dd3bd...,1,2,37,0,0,0,0,0,"[1, 2, 37]"
1,121,6-avi-frame6431_jpg.rf.1ad48ac0ce545b88cefb946...,8,0,0,0,0,0,0,0,[8]
2,1130,9-avi-frame1457_jpg.rf.634a979898a9caa4d106913...,21,0,0,0,0,0,0,0,[21]
3,378,3-avi-frame499_jpg.rf.a0464ac21591528194da5bcf...,5,4,0,0,0,0,0,0,"[5, 4]"
4,1079,5-avi-frame5306_jpg.rf.8cd8b5643eea33f5f33714e...,48,0,0,0,0,0,0,0,[48]
...,...,...,...,...,...,...,...,...,...,...,...
773,777,6-avi-frame4576_jpg.rf.8780eec122abf32efb3744c...,8,0,0,0,0,0,0,0,[8]
774,739,5-avi-frame3664_jpg.rf.71c9b545a181651166de56a...,8,0,0,0,0,0,0,0,[8]
775,873,6-avi-frame6490_jpg.rf.b19dac7915db559d41e7e27...,8,4,5,0,0,0,0,0,"[8, 4, 5]"
776,173,5-avi-frame4887_jpg.rf.9a87ce8aac43977d561d819...,38,51,0,0,0,0,0,0,"[38, 51]"


In [16]:
train_data['sing_class'][777] = [2]

C:\Users\LaughingMan\AppData\Local\Temp\ipykernel_20356\3177502563.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_data['sing_class'][777] = [2]


In [17]:
train_data

,id,img,sing1,sing2,sing3,sing4,sing5,sing6,sing7,sing8,sing_class
0,807,5-avi-frame24_jpg.rf.5dec372f9195e9a88ff7dd3bd...,1,2,37,0,0,0,0,0,"[1, 2, 37]"
1,121,6-avi-frame6431_jpg.rf.1ad48ac0ce545b88cefb946...,8,0,0,0,0,0,0,0,[8]
2,1130,9-avi-frame1457_jpg.rf.634a979898a9caa4d106913...,21,0,0,0,0,0,0,0,[21]
3,378,3-avi-frame499_jpg.rf.a0464ac21591528194da5bcf...,5,4,0,0,0,0,0,0,"[5, 4]"
4,1079,5-avi-frame5306_jpg.rf.8cd8b5643eea33f5f33714e...,48,0,0,0,0,0,0,0,[48]
...,...,...,...,...,...,...,...,...,...,...,...
773,777,6-avi-frame4576_jpg.rf.8780eec122abf32efb3744c...,8,0,0,0,0,0,0,0,[8]
774,739,5-avi-frame3664_jpg.rf.71c9b545a181651166de56a...,8,0,0,0,0,0,0,0,[8]
775,873,6-avi-frame6490_jpg.rf.b19dac7915db559d41e7e27...,8,4,5,0,0,0,0,0,"[8, 4, 5]"
776,173,5-avi-frame4887_jpg.rf.9a87ce8aac43977d561d819...,38,51,0,0,0,0,0,0,"[38, 51]"


In [18]:
NUM_CLASSES = 69

In [19]:
generator = ImageDataGenerator(
    preprocessing_function=preprocessing_function
)
backbone = ResNet50(weights='imagenet', include_top=False)
x = backbone.output
x = GlobalAveragePooling2D()(x)
x = Dense('300', activation='relu6')(x)
predictions = Dense(NUM_CLASSES, activation='sigmoid')(x)
model = Model(inputs=backbone.input, outputs=predictions)
opt = tf.keras.optimizers.Adam(learning_rate=0.001)
model.compile(
    optimizer=opt, 
    loss='categorical_crossentropy', 
    metrics=[
      tf.keras.metrics.CategoricalAccuracy(),
      tf.keras.metrics.Precision(), 
      tf.keras.metrics.Recall(),
      tf.keras.metrics.AUC()
    ]
)

In [20]:
train_data

,id,img,sing1,sing2,sing3,sing4,sing5,sing6,sing7,sing8,sing_class
0,807,5-avi-frame24_jpg.rf.5dec372f9195e9a88ff7dd3bd...,1,2,37,0,0,0,0,0,"[1, 2, 37]"
1,121,6-avi-frame6431_jpg.rf.1ad48ac0ce545b88cefb946...,8,0,0,0,0,0,0,0,[8]
2,1130,9-avi-frame1457_jpg.rf.634a979898a9caa4d106913...,21,0,0,0,0,0,0,0,[21]
3,378,3-avi-frame499_jpg.rf.a0464ac21591528194da5bcf...,5,4,0,0,0,0,0,0,"[5, 4]"
4,1079,5-avi-frame5306_jpg.rf.8cd8b5643eea33f5f33714e...,48,0,0,0,0,0,0,0,[48]
...,...,...,...,...,...,...,...,...,...,...,...
773,777,6-avi-frame4576_jpg.rf.8780eec122abf32efb3744c...,8,0,0,0,0,0,0,0,[8]
774,739,5-avi-frame3664_jpg.rf.71c9b545a181651166de56a...,8,0,0,0,0,0,0,0,[8]
775,873,6-avi-frame6490_jpg.rf.b19dac7915db559d41e7e27...,8,4,5,0,0,0,0,0,"[8, 4, 5]"
776,173,5-avi-frame4887_jpg.rf.9a87ce8aac43977d561d819...,38,51,0,0,0,0,0,0,"[38, 51]"


In [21]:

train_data, test_data = train_test_split(train_data, test_size=0.2)

In [22]:
classes.values()

dict_values(['5_18', '3_24', '1_16', '5_15_5', '5_19_1', '5_19_2', '1_20_1', '8_23', '2_1', '4_2_1', '8_22_1', '6_16', '1_22', '1_2', '5_16', '3_27', '6_10_1', '8_2_4', '6_12', '5_15_2', '3_13', '3_1', '3_20', '3_12', '7_14_2', '5_23_1', '2_4', '5_6', '4_2_3', '8_22_3', '5_15_1', '7_3', '3', '2_3_1', '3_11', '6_13', '5_15_4', '8_2_1', '1_34_3', '8_2_2', '5_15_3', '1_17', '4_1_1', '4_1_4', '3_25', '1_20_2', '8_22_2', '6_9_2', '3_2', '5_5', '5_15_7', '7_12', '8_2_3', '5_24_1', '1_25', '3_28', '5_9_1', '5_15_6', '8_1_1', '1_10', '6_11', '3_4', '6_10', '6_9_1', '8_2_5', '5_15', '4_8_2', '8_22', '5_21'])

In [23]:
train_gen = generator.flow_from_dataframe(
    dataframe=train_data,
    directory='E:/AutoPilotChampionship/data/trainset/train/',
    x_col = 'img',
    y_col = 'sing_class',
    classes=classes.keys(),
    class_mode = 'categorical',
    target_size = (224, 224),
    color_mode = 'rgb',
    batch_size = 16,
    validate_filenames=False
)

Found 622 non-validated image filenames belonging to 69 classes.


In [24]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

In [25]:
train_gen.class_indices.keys()

dict_keys([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68])

In [26]:
test_gen = generator.flow_from_dataframe(
    dataframe=test_data,
    directory='E:/AutoPilotChampionship/data/testset/test/',
    x_col = 'img',
    y_col = 'sing_class',
    classes= train_gen.class_indices.values(),
    class_mode = 'categorical',
    target_size = (224, 224),
    color_mode = 'rgb',
    batch_size = 16,
    validate_filenames=False
) 

Found 156 non-validated image filenames belonging to 69 classes.


In [27]:
tf.test.is_built_with_cuda()

True

In [28]:
from keras import backend as K
K.clear_session()


In [29]:
STEP_SIZE_TRAIN= train_gen.n//train_gen.batch_size
STEP_SIZE_TEST= test_gen.n//test_gen.batch_size
model.fit(train_gen,
    steps_per_epoch=STEP_SIZE_TRAIN,
    validation_data=test_gen,
    validation_steps=STEP_SIZE_TEST,
    epochs=4
)

Epoch 1/4
 8/38 [=====>........................] - ETA: 14s - loss: 11.1037 - categorical_accuracy: 0.2460 - precision: 0.0578 - recall: 0.8000 - auc: 0.7824

ResourceExhaustedError: Graph execution error:

Detected at node 'model/conv5_block3_3_conv/Conv2D' defined at (most recent call last):
    File "c:\Users\LaughingMan\anaconda3\envs\autopilot\lib\runpy.py", line 197, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "c:\Users\LaughingMan\anaconda3\envs\autopilot\lib\runpy.py", line 87, in _run_code
      exec(code, run_globals)
    File "c:\Users\LaughingMan\anaconda3\envs\autopilot\lib\site-packages\ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "c:\Users\LaughingMan\anaconda3\envs\autopilot\lib\site-packages\traitlets\config\application.py", line 978, in launch_instance
      app.start()
    File "c:\Users\LaughingMan\anaconda3\envs\autopilot\lib\site-packages\ipykernel\kernelapp.py", line 712, in start
      self.io_loop.start()
    File "c:\Users\LaughingMan\anaconda3\envs\autopilot\lib\site-packages\tornado\platform\asyncio.py", line 215, in start
      self.asy